In [1]:
!nvidia-smi

Wed Mar 22 00:57:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install pycuda scikit-cuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.2 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp39-cp39-linux_x86_64.whl size=661963 sha256=4439a32370e28f2cca57c27d54d5bb6a907f92f6abb069c63025353f9857268b
  Stored in directory: /root/.cache/pip/wheels/82/09/27/74d8e31ed19c530166e0d263aabe1ea57465e255615bda8fc0
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69866 sha256=8ba865ad6f95411d8e50a3da8daa597bb83770f

In [31]:
from __future__ import division
import pycuda.autoinit
from pycuda import gpuarray
import numpy as np
from skcuda import linalg
from matplotlib import pyplot as plt

import PIL
import urllib
from time import time

In [103]:
m = 2800
n = 4500
k = 30

In [104]:
B = np.random.randn(m, k).astype(np.float32)
C = np.random.randn(k, n).astype(np.float32)
A = np.matmul(B, C)
print(A.shape)
A_gpu = gpuarray.to_gpu(A)

(2800, 4500)


In [105]:
t=time()
U,S,V=np.linalg.svd(A, full_matrices=False)
t=time()-t
print(f'Ellapsed time: {t}')

Ellapsed time: 27.56638193130493


In [106]:
print(U.shape)
print(S.shape)
print(V.shape)

(2800, 2800)
(2800,)
(2800, 4500)


In [107]:
Ahat=np.matmul(np.matmul(U,np.diag(S)),V)

In [109]:
np.max(np.abs(A-Ahat))

1.1444092e-05

In [110]:
np.allclose(A,Ahat,atol=1e-2)

True

In [111]:
linalg.init()

In [112]:
t=time()
U_gpu,S_gpu,V_gpu = linalg.svd(A_gpu,'S','S',lib='cusolver') 
t=time()-t
print(f'Ellapsed time: {t}')

Ellapsed time: 12.056218147277832


In [113]:
U2=U_gpu.get()
S2=S_gpu.get()
V2=V_gpu.get()

In [114]:
print(U2.shape)
print(S2.shape)
print(V2.shape)

(2800, 2800)
(2800,)
(2800, 4500)


In [115]:
Ahat2=np.matmul(np.matmul(U2,np.diag(S2)),V2)
print(np.max(np.abs(A-Ahat2)))

0.00012874603


In [116]:
np.allclose(A,Ahat2,atol=1e-2)

True